In [23]:
textgrid_dir = "/home/joregan/textgrids"
wav_dir = "/home/joregan/hsi/audio/"

In [2]:
from pydub import AudioSegment

In [26]:
from pathlib import Path

textgrid_path = Path(textgrid_dir)
wav_path = Path(wav_dir)

In [5]:
from praatio import textgrid

In [6]:
import re

def noise_event(text):
    m = re.match("^\[([^\]]+)\]$", text)
    return (m is not None)

In [7]:
def get_tier_entries(textgrid, tiername="utterances"):
    entries = []

    tier = textgrid.getTier(tiername)
    for entry in tier.entries:
        text = entry.label
        if text.strip() == "":
            continue
        if not noise_event(text.strip()):
            entries.append({
                "start": entry.start,
                "end": entry.end,
                "text": entry.label
            })
    return entries

In [8]:
def run_espeak(text):
    phon = !echo "{text}" | espeak-ng -v en-us --ipa -q
    return (" ".join(phon)).strip()

In [9]:
def espeakify_entries(entries):
    for entry in entries:
        entry["espeak"] = run_espeak(entry["text"])

In [73]:
import numpy as np

def entries_add_audio(wavfile, entries):
    audio = AudioSegment.from_file(wavfile)
    dtype = getattr(np, "int{:d}".format(audio.sample_width * 8))

    for entry in entries:
        start = int(entry["start"] * 1000)
        end = int(entry["end"] * 1000)
        selection = audio[start:end]
        # selection = selection.set_frame_rate(16000)
        array = np.ndarray((int(selection.frame_count()), selection.channels), buffer=selection.raw_data, dtype=dtype).astype("float32")
        array /= np.iinfo(dtype).max
        entry["audio"] = {
            "array": array.squeeze(),
            "sampling_rate": audio.frame_rate
        }
        

In [12]:
from transformers import AutoProcessor, AutoModelForCTC, Wav2Vec2Processor
import torch
from itertools import groupby

def decode_phonemes(
    ids: torch.Tensor, processor: Wav2Vec2Processor, ignore_stress: bool = False
) -> str:
    """CTC-like decoding. First removes consecutive duplicates, then removes special tokens."""
    # removes consecutive duplicates
    ids = [id_ for id_, _ in groupby(ids)]

    special_token_ids = processor.tokenizer.all_special_ids + [
        processor.tokenizer.word_delimiter_token_id
    ]
    # converts id to token, skipping special tokens
    phonemes = [processor.decode(id_) for id_ in ids if id_ not in special_token_ids]

    # joins phonemes
    prediction = " ".join(phonemes)

    # whether to ignore IPA stress marks
    if ignore_stress == True:
        prediction = prediction.replace("ˈ", "").replace("ˌ", "")

    return prediction

checkpoint = "bookbot/wav2vec2-ljspeech-gruut"

model = AutoModelForCTC.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)
sr = processor.feature_extractor.sampling_rate

/home/joregan/miniconda3/envs/hf_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
entries = []
for tgfile in textgrid_path.glob("*.[Tt]ext[Gg]rid"):
    wavname = f"{tgfile.stem}.wav"
    tg = textgrid.openTextgrid(tgfile, False)
    entries = get_tier_entries(tg)
    espeakify_entries(entries)
    entries_add_audio(str(wav_path / wavname), entries)
    print(entries)



In [74]:
from datasets import Dataset

wavname = "/home/joregan/hsi/audio/hsi_7_0719_222_002_inter.wav"
tgfile = "/home/joregan/textgrids/hsi_7_0719_222_002_inter.TextGrid"
tg = textgrid.openTextgrid(tgfile, False)
entries = get_tier_entries(tg)
espeakify_entries(entries)
entries_add_audio(wavname, entries)
entry_ds = Dataset.from_list(entries)

In [ ]:
entry_ds[0]["audio"]["array"]

In [77]:
from datasets import Audio

entry_ds.cast_column("audio", Audio(sampling_rate=16000))

Dataset({
    features: ['start', 'end', 'text', 'espeak', 'audio'],
    num_rows: 101
})

In [78]:
inputs = processor(entry_ds["audio"], sampling_rate=16_000, return_tensors="pt", padding=True)


ValueError: type of {'array': [0.0005220388993620872, -0.0005438695661723614, -0.0008447538129985332, 9.017018601298332e-05, -0.0005115978419780731, -0.0010644844733178616, -0.0010436028242111206, 0.00040054600685834885, -0.0002159341238439083, -0.0010720780119299889, -0.0013900469057261944, 0.000167052261531353, -0.0005291574634611607, -0.0007303799502551556, -5.837343633174896e-05, 0.00025579892098903656, -0.0009852293878793716, -0.0003635287284851074, 7.11856409907341e-06, -0.000574242789298296, -0.0007550581358373165, -0.0005429205484688282, -0.0005054282955825329, -0.0006691589951515198, -0.00032888446003198624, -0.0010170266032218933, -0.0004598689265549183, -0.0004926146939396858, 0.0001328825019299984, -0.0009287544526159763, -0.0006060395389795303, -3.322027623653412e-06, -0.0006311922334134579, -0.000200747512280941, -2.5152694433927536e-05, -0.0009904499165713787, -0.0006515993736684322, 5.505140870809555e-05, 9.396718814969063e-05, -0.0004332922399044037, 0.0001784423366189